# Exploración de datos COVID-19

1. Habilidades SQL utilizadas: Uniones, CTE's, Tablas Temporales, Funciones Windows, Funciones Agregadas, Creación de Vistas, Conversión de Tipos de Datos

2. Hablilidades GCP utilizadas: Big Query

3. Habilidades Python utilizadas: Pandas, Manipular GCP desde codigo

## 1.Obtener la informacion
Lo primero que debemos realizar es tener montada nuestra informacion en BigQuery, encontre el siguiente link: [Dataset](https://ourworldindata.org/covid-deaths), Este es un conjunto de datos que proporciona información sobre el número de muertes confirmadas por COVID-19 y el número de vacunaciones COVID-19 confirmadas.

- Lo primero que se realizó fue separar el conjunto de datos en 2 partes. Para esto, simplemente apliqué un análisis en Excel reconociendo las columnas de datos referentes a vacunaciones y muertes para poder hacer la separación.

- Lo segundo que se debe hacer es ir directamente a GCP e ir al apartado de BigQuery y crear un conjunto de datos llamado "CovidDB". Luego debemos crear 2 tablas, una llamada "CovidDeaths" donde esta hace referencia a nuestra tabla dividida en excel "CovidDeaths.csv" y asi mismo con la otra llamada "CovidVaccinations" haciendo referencia a "CovidVaccinations.csv".

**Nota:** Tecnicamente desde el portal de BigQuery se pueden realizar las querys, pero este proyecto consiste en unicar el uso de Python con GCP ademas Python es una excelente opción para trabajar con GCP debido a su SDK específico para Python, su amplia variedad de bibliotecas y marcos, su flexibilidad y la comunidad activa de desarrolladores.

## 2.Autentificarse en GCP


In [55]:
from google.colab import auth
auth.authenticate_user()

##### Especifico qué project_id vamos a utilizar. 

In [56]:
project_id = 'covidproyectjf'

## 3.Conectarse a la API de BigQuery

In [57]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

## 4.Acceso al dataset (público o privado)
Ponemos una referencia para el dataset

In [58]:
dataset_ref = client.dataset("CovidDB",project="covidproyectjf")
dataset = client.get_dataset(dataset_ref)

## 5.Identificar las tablas



In [59]:
tables = list (client.list_tables(dataset))
for table in tables:
  print(table.table_id)

CovidDeaths
CovidVaccinations
PercentPopulationVaccinated


## 6.Comprobar el esquema de la tabla

In [60]:
covid_d_table_ref = dataset_ref.table('CovidDeaths')
covid_v_table_ref = dataset_ref.table('CovidVaccinations')
covid_death_table = client.get_table(covid_d_table_ref)
covid_vaccinations_table = client.get_table(covid_v_table_ref)

In [61]:
covid_death_table.schema

[SchemaField('iso_code', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('continent', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('location', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('date', 'DATE', 'NULLABLE', None, None, (), None),
 SchemaField('total_cases', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('new_cases', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('new_cases_smoothed', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('total_deaths', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('new_deaths', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('new_deaths_smoothed', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('total_cases_per_million', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('new_cases_per_million', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('new_cases_smoothed_per_million', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('t

In [62]:
covid_vaccinations_table.schema

[SchemaField('iso_code', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('continent', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('location', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('date', 'DATE', 'NULLABLE', None, None, (), None),
 SchemaField('new_tests', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('total_tests', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('total_tests_per_thousand', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('new_tests_per_thousand', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('new_tests_smoothed', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('new_tests_smoothed_per_thousand', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('positive_rate', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('tests_per_case', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('tests_units', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('to

## 7.Muestro la informacion obtenida en un dataframe

In [63]:
df_covid_d = client.list_rows(covid_death_table).to_dataframe()
df_covid_d

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,OWID_AFR,,Africa,2020-02-13,<NA>,0,NaN,<NA>,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OWID_AFR,,Africa,2020-02-14,1,1,NaN,<NA>,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OWID_AFR,,Africa,2020-02-15,1,0,NaN,<NA>,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OWID_AFR,,Africa,2020-02-16,1,0,NaN,<NA>,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OWID_AFR,,Africa,2020-02-17,1,0,NaN,<NA>,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85166,SAU,Asia,Saudi Arabia,2021-04-26,413174,958,1033.429,6913,13,11.286,...,49045.411,NaN,259.538,17.72,1.8,25.4,NaN,2.7,75.13,0.854
85167,SAU,Asia,Saudi Arabia,2021-04-27,414219,1045,1029.857,6922,9,10.857,...,49045.411,NaN,259.538,17.72,1.8,25.4,NaN,2.7,75.13,0.854
85168,SAU,Asia,Saudi Arabia,2021-04-28,415281,1062,1034.714,6935,13,11.000,...,49045.411,NaN,259.538,17.72,1.8,25.4,NaN,2.7,75.13,0.854
85169,SAU,Asia,Saudi Arabia,2021-04-29,416307,1026,1030.571,6946,11,11.000,...,49045.411,NaN,259.538,17.72,1.8,25.4,NaN,2.7,75.13,0.854


In [64]:
df_covid_v = client.list_rows(covid_vaccinations_table).to_dataframe()
df_covid_v

,iso_code,continent,location,date,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,OWID_AFR,None,Africa,2020-02-13,<NA>,<NA>,NaN,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OWID_AFR,None,Africa,2020-02-14,<NA>,<NA>,NaN,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OWID_AFR,None,Africa,2020-02-15,<NA>,<NA>,NaN,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OWID_AFR,None,Africa,2020-02-16,<NA>,<NA>,NaN,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OWID_AFR,None,Africa,2020-02-17,<NA>,<NA>,NaN,NaN,<NA>,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85166,URY,South America,Uruguay,2021-04-26,10927,1745442,502.470,3.146,14798,4.260,...,20551.409,0.1,160.708,6.93,14.0,19.9,NaN,2.8,77.91,0.817
85167,URY,South America,Uruguay,2021-04-27,14368,1759810,506.606,4.136,14554,4.190,...,20551.409,0.1,160.708,6.93,14.0,19.9,NaN,2.8,77.91,0.817
85168,URY,South America,Uruguay,2021-04-28,15171,1774981,510.973,4.367,14201,4.088,...,20551.409,0.1,160.708,6.93,14.0,19.9,NaN,2.8,77.91,0.817
85169,URY,South America,Uruguay,2021-04-29,14347,1789328,515.103,4.130,14106,4.061,...,20551.409,0.1,160.708,6.93,14.0,19.9,NaN,2.8,77.91,0.817


## 8.- Empezamos con una query de test para verificar si podemos ponernos creativos!!!

In [65]:
query = """
SELECT Location, date, total_cases, new_cases, total_deaths, population
FROM CovidDB.CovidDeaths
WHERE continent IS NOT NULL 
"""
df_test = client.query(query).to_dataframe()
df_test

,Location,date,total_cases,new_cases,total_deaths,population
0,Africa,2020-02-13,<NA>,0,<NA>,1340598113
1,Africa,2020-02-14,1,1,<NA>,1340598113
2,Africa,2020-02-15,1,0,<NA>,1340598113
3,Africa,2020-02-16,1,0,<NA>,1340598113
4,Africa,2020-02-17,1,0,<NA>,1340598113
...,...,...,...,...,...,...
85166,Saudi Arabia,2021-04-26,413174,958,6913,34813867
85167,Saudi Arabia,2021-04-27,414219,1045,6922,34813867
85168,Saudi Arabia,2021-04-28,415281,1062,6935,34813867
85169,Saudi Arabia,2021-04-29,416307,1026,6946,34813867


### Total de casos frente a total de muertes
Muestra la probabilidad de morir si contraes covid en tu país

In [87]:
query = """
SELECT Location, date, total_cases,total_deaths, (total_deaths/total_cases)*100 AS DeathPercentage
FROM CovidDB.CovidDeaths
WHERE total_deaths IS NOT NULL
ORDER BY 1,2
"""
df1 = client.query(query).to_dataframe()
df1

,Location,date,total_cases,total_deaths,DeathPercentage
0,Afghanistan,2020-03-22,34,1,2.941176
1,Afghanistan,2020-03-23,41,1,2.439024
2,Afghanistan,2020-03-24,43,1,2.325581
3,Afghanistan,2020-03-25,76,2,2.631579
4,Afghanistan,2020-03-26,80,3,3.750000
...,...,...,...,...,...
73403,Zimbabwe,2021-04-26,38102,1560,4.094273
73404,Zimbabwe,2021-04-27,38164,1565,4.100723
73405,Zimbabwe,2021-04-28,38191,1565,4.097824
73406,Zimbabwe,2021-04-29,38235,1567,4.098339


### Mexico

In [67]:
df1_mex = df1[df1['Location'] == 'Mexico'].dropna()
df1_mex

,Location,date,total_cases,total_deaths,DeathPercentage
49563,Mexico,2020-03-19,164,1,0.609756
49564,Mexico,2020-03-20,203,2,0.985222
49565,Mexico,2020-03-21,251,2,0.796813
49566,Mexico,2020-03-22,316,3,0.949367
49567,Mexico,2020-03-23,367,4,1.089918
...,...,...,...,...,...
49966,Mexico,2021-04-26,2329534,215113,9.234164
49967,Mexico,2021-04-27,2333126,215547,9.238549
49968,Mexico,2021-04-28,2336944,215918,9.239331
49969,Mexico,2021-04-29,2340934,216447,9.246181


### Casos totales frente a población
Muestra el porcentaje de población infectada por Covid

In [68]:
query = """
SELECT Location, date, Population, total_cases,  (total_cases/population)*100 AS PercentPopulationInfected
FROM CovidDB.CovidDeaths
ORDER BY 1,2
"""
df2 = client.query(query).to_dataframe()
df2

,Location,date,Population,total_cases,PercentPopulationInfected
0,Afghanistan,2020-02-24,38928341,1,0.000003
1,Afghanistan,2020-02-25,38928341,1,0.000003
2,Afghanistan,2020-02-26,38928341,1,0.000003
3,Afghanistan,2020-02-27,38928341,1,0.000003
4,Afghanistan,2020-02-28,38928341,1,0.000003
...,...,...,...,...,...
85166,Zimbabwe,2021-04-26,14862927,38102,0.256356
85167,Zimbabwe,2021-04-27,14862927,38164,0.256773
85168,Zimbabwe,2021-04-28,14862927,38191,0.256955
85169,Zimbabwe,2021-04-29,14862927,38235,0.257251


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


### En Mexico

In [69]:
df2_mex = df2[df2['Location'] == 'Mexico'].dropna()
df2_mex

,Location,date,Population,total_cases,PercentPopulationInfected
49543,Mexico,2020-02-28,128932753,1,7.755981e-07
49544,Mexico,2020-02-29,128932753,4,3.102392e-06
49545,Mexico,2020-03-01,128932753,5,3.877991e-06
49546,Mexico,2020-03-02,128932753,5,3.877991e-06
49547,Mexico,2020-03-03,128932753,5,3.877991e-06
...,...,...,...,...,...
49966,Mexico,2021-04-26,128932753,2329534,1.806782e+00
49967,Mexico,2021-04-27,128932753,2333126,1.809568e+00
49968,Mexico,2021-04-28,128932753,2336944,1.812529e+00
49969,Mexico,2021-04-29,128932753,2340934,1.815624e+00


### Top 20 Países con mayor tasa de infección en comparación con la población

In [82]:
query = """
SELECT Location, Population, MAX(total_cases) AS HighestInfectionCount,  MAX((total_cases/population))*100 AS PercentPopulationInfected
FROM CovidDB.CovidDeaths
GROUP BY Location, Population
ORDER BY PercentPopulationInfected DESC
LIMIT 20
"""
df3 = client.query(query).to_dataframe()
df3

,Location,Population,HighestInfectionCount,PercentPopulationInfected
0,Andorra,77265,13232,17.125477
1,Montenegro,628062,97389,15.506272
2,Czechia,10708982,1630758,15.227946
3,San Marino,33938,5066,14.927220
4,Slovenia,2078932,240292,11.558435
5,Luxembourg,625976,67205,10.736035
6,Bahrain,1701583,176934,10.398200
7,Serbia,6804596,689557,10.133695
8,United States,331002647,32346971,9.772421
9,Israel,8655541,838481,9.687217


### En Mexico

In [71]:
df3_mex = df3[df3['Location'] == 'Mexico'].dropna()
df3_mex

,Location,Population,HighestInfectionCount,PercentPopulationInfected
92,Mexico,128932753,2344755,1.818588


### TOP 20 Países con mayor número de muertes por población

In [96]:
query = """
SELECT Location, MAX(CAST(Total_deaths AS int)) AS TotalDeathCount
FROM CovidDB.CovidDeaths
WHERE continent IS NOT NULL
AND Location != "World"
GROUP BY Location
ORDER BY TotalDeathCount DESC
LIMIT 20
"""
df4 = client.query(query).to_dataframe()
df4

,Location,TotalDeathCount
0,Europe,1016750
1,North America,847942
2,European Union,688896
3,South America,672415
4,United States,576232
5,Asia,520286
6,Brazil,403781
7,Mexico,216907
8,India,211853
9,United Kingdom,127775


### DESGLOSANDO LAS COSAS POR CONTINENTE
Mostrando los continentes con mayor número de muertes por población


In [97]:
query = """
SELECT continent, MAX(cast(Total_deaths AS int)) AS TotalDeathCount
FROM CovidDB.CovidDeaths
WHERE continent IS NOT NULL
AND continent != "" 
GROUP BY continent
ORDER BY TotalDeathCount DESC
"""
df5 = client.query(query).to_dataframe()
df5

,continent,TotalDeathCount
0,North America,576232
1,South America,403781
2,Asia,211853
3,Europe,127775
4,Africa,54350
5,Oceania,910


### CIFRAS GLOBALES

In [75]:
query = """
SELECT SUM(new_cases) AS total_cases, SUM(cast(new_deaths AS int)) AS total_deaths, SUM(CAST(new_deaths as int))/SUM(New_Cases)*100 AS DeathPercentage
From CovidDB.CovidDeaths
WHERE continent IS NOT NULL
ORDER BY 1,2;
"""
df6 = client.query(query).to_dataframe()
df6

,total_cases,total_deaths,DeathPercentage
0,482497587,10229544,2.120123


### Población total frente a vacunaciones
Muestra el porcentaje de población que ha recibido al menos una vacuna Covid

In [108]:
query = """
SELECT dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations, 
SUM(CAST(vac.new_vaccinations AS INT)) OVER (PARTITION BY dea.location ORDER BY dea.location, dea.date) AS RollingPeopleVaccinated
FROM CovidDB.CovidDeaths dea
JOIN CovidDB.CovidVaccinations vac
	ON dea.location = vac.location
	AND dea.date = vac.date
WHERE dea.continent IS NOT NULL
AND vac.new_vaccinations IS NOT NULL
ORDER BY 2,3;
"""
df7 = client.query(query).to_dataframe()
df7

,continent,location,date,population,new_vaccinations,RollingPeopleVaccinated
0,,Africa,2021-01-16,1340598113,3000,3000
1,,Africa,2021-01-20,1340598113,2187,5187
2,,Africa,2021-01-23,1340598113,1463,6650
3,,Africa,2021-01-24,1340598113,0,6650
4,,Africa,2021-01-25,1340598113,1536,8186
...,...,...,...,...,...,...
7949,Africa,Zimbabwe,2021-04-26,14862927,21592,410296
7950,Africa,Zimbabwe,2021-04-27,14862927,22329,432625
7951,Africa,Zimbabwe,2021-04-28,14862927,24074,456699
7952,Africa,Zimbabwe,2021-04-29,14862927,19584,476283


### En Mexico

In [77]:
df7_mex = df7[df7['location'] == 'Mexico'].dropna()
df7_mex

,continent,location,date,population,new_vaccinations,RollingPeopleVaccinated
49847,North America,Mexico,2020-12-28,128932753,2755,2755
49848,North America,Mexico,2020-12-29,128932753,8950,11705
49849,North America,Mexico,2020-12-30,128932753,6469,18174
49855,North America,Mexico,2021-01-05,128932753,4276,22450
49856,North America,Mexico,2021-01-06,128932753,4949,27399
...,...,...,...,...,...,...
49964,North America,Mexico,2021-04-24,128932753,339675,15365049
49965,North America,Mexico,2021-04-25,128932753,195550,15560599
49966,North America,Mexico,2021-04-26,128932753,91705,15652304
49967,North America,Mexico,2021-04-27,128932753,185450,15837754


### Utilización de CTE para realizar cálculos sobre Partition By en la consulta anterior

In [112]:
query = """
WITH PopvsVac 
AS
(
  SELECT dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations, 
  SUM(CAST(vac.new_vaccinations AS INT)) OVER (PARTITION BY dea.Location ORDER BY dea.location, dea.Date) AS RollingPeopleVaccinated
  FROM CovidDB.CovidDeaths dea
  JOIN CovidDB.CovidVaccinations vac
	  ON dea.location = vac.location
	  AND dea.date = vac.date
  WHERE dea.continent IS NOT NULL
)
Select *, (RollingPeopleVaccinated/Population)*100
From PopvsVac
WHERE RollingPeopleVaccinated/Population IS NOT NULL;
"""
df8 = client.query(query).to_dataframe()
df8

,continent,location,date,population,new_vaccinations,RollingPeopleVaccinated,f0_
0,,Africa,2021-01-16,1340598113,3000,3000,0.000224
1,,Africa,2021-01-17,1340598113,<NA>,3000,0.000224
2,,Africa,2021-01-18,1340598113,<NA>,3000,0.000224
3,,Africa,2021-01-19,1340598113,<NA>,3000,0.000224
4,,Africa,2021-01-20,1340598113,2187,5187,0.000387
...,...,...,...,...,...,...,...
11114,Africa,Zimbabwe,2021-04-26,14862927,21592,410296,2.760533
11115,Africa,Zimbabwe,2021-04-27,14862927,22329,432625,2.910766
11116,Africa,Zimbabwe,2021-04-28,14862927,24074,456699,3.072739
11117,Africa,Zimbabwe,2021-04-29,14862927,19584,476283,3.204503


### En Mexico

In [79]:
df8_mex = df8[df8['location'] == 'Mexico'].dropna()
df8_mex

,continent,location,date,population,new_vaccinations,RollingPeopleVaccinated,f0_
49847,North America,Mexico,2020-12-28,128932753,2755,2755,0.002137
49848,North America,Mexico,2020-12-29,128932753,8950,11705,0.009078
49849,North America,Mexico,2020-12-30,128932753,6469,18174,0.014096
49855,North America,Mexico,2021-01-05,128932753,4276,22450,0.017412
49856,North America,Mexico,2021-01-06,128932753,4949,27399,0.021251
...,...,...,...,...,...,...,...
49964,North America,Mexico,2021-04-24,128932753,339675,15365049,11.917103
49965,North America,Mexico,2021-04-25,128932753,195550,15560599,12.068771
49966,North America,Mexico,2021-04-26,128932753,91705,15652304,12.139897
49967,North America,Mexico,2021-04-27,128932753,185450,15837754,12.283732


### Utilización de la tabla para realizar cálculos sobre Partition By en la consulta anterior

In [111]:
query_a = """
DROP TABLE IF EXISTS PercentPopulationVaccinated
"""

query_b = """
CREATE TABLE IF NOT EXISTS CovidDB.PercentPopulationVaccinated
AS (
  SELECT 
    dea.continent, 
    dea.location, 
    DATETIME(dea.date) AS Date, 
    dea.population, 
    vac.new_vaccinations,
    SUM(CAST(vac.new_vaccinations AS INT)) OVER (
      PARTITION BY dea.Location ORDER BY dea.location, dea.Date
    ) AS RollingPeopleVaccinated
  FROM CovidDB.CovidDeaths dea
  JOIN CovidDB.CovidVaccinations vac
    ON dea.location = vac.location
    AND dea.date = vac.date
);
"""

query_c = """
SELECT *, (RollingPeopleVaccinated/Population)*100
FROM CovidDB.PercentPopulationVaccinated
WHERE RollingPeopleVaccinated/Population IS NOT NULL;
"""

# Ejecutar consulta_a para eliminar la tabla temporal existente, si existe
client.query(query_a)

# Ejecutar consulta_b para crear la tabla temporal
client.query(query_b)

# Ejecutar consulta_c para seleccionar los datos de la tabla temporal y calcular el porcentaje de la población vacunada
df9 = client.query(query_c).to_dataframe()
df9

,continent,location,Date,population,new_vaccinations,RollingPeopleVaccinated,f0_
0,North America,Aruba,2021-04-27,106766,<NA>,30287,28.367645
1,North America,Bahamas,2021-04-18,393248,<NA>,6890,1.752075
2,North America,Bahamas,2021-04-22,393248,<NA>,6890,1.752075
3,Asia,Bhutan,2021-03-28,771612,97322,183271,23.751704
4,Africa,Cote d'Ivoire,2021-04-15,26378275,3842,59204,0.224442
...,...,...,...,...,...,...,...
11114,Europe,Ukraine,2021-03-02,43733759,1983,4965,0.011353
11115,Europe,Ukraine,2021-03-12,43733759,9304,45618,0.104308
11116,Europe,Ukraine,2021-04-11,43733759,1216,286030,0.654026
11117,Europe,Ukraine,2021-02-26,43733759,1713,2892,0.006613
